In [2]:
import robosandbox as rsb
import numpy as np
import swift
import os

## Generic Models

In [3]:
qlim_l = np.array([-np.pi, -np.pi, -np.pi, -np.pi])
qlim_u = np.array([np.pi, np.pi, np.pi, np.pi])
qlim = np.vstack((qlim_l, qlim_u))
G4 = rsb.models.URDF.GenericDH(
    dofs=4,
    a=[0, -0.4, -0.4, -0.4],
    d=[0.4, 0, 0, 0],
    alpha=[np.pi / 2, 0, 0, 0],
    qlim=qlim,
)
G4.q = np.array([np.pi / 2, -np.pi / 2, 0, 0])


axis is [-0.  0.  0.]
angle is 0.0
axis is [ 0. -1.  0.]
angle is 1.5707963267948966
axis is [ 0. -1.  0.]
angle is 1.5707963267948966
axis is [ 0. -1.  0.]
angle is 1.5707963267948966


/var/folders/x1/r8g1dmlj1dn7lz6z4bdx_mfr0000gn/T/ipykernel_90846/3889713405.py:4: UserWarning:

Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.



## Commercial models

### Panda

In [4]:
panda = rsb.models.URDF.Panda()
panda.q = panda.qz
env = swift.Swift()
env.launch(realtime=True, browser="notebook")
env.add(panda)

0

### Puma560

In [5]:
puma = rsb.models.URDF.Puma560()
puma.q = puma.qr
env = swift.Swift()
env.launch(realtime=True, browser="notebook")
env.add(puma)

0

## Read from URDF File

In [10]:
# get the current working directory
pwd = os.getcwd()
tld = os.path.join(pwd, "data", "SO101")

SO101 = rsb.models.URDF.Model(
    name="SO101",
    file_path="so101_new_calib.urdf",
    tld=tld,
)
SO101.q = np.zeros(6)
# print(SO101)

env = swift.Swift()
env.launch(realtime=True, browser="notebook")
env.add(SO101)


# SO101.plot(q=SO101.q)

0

In [4]:
print(puma._T)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [70]:
env = swift.Swift()
env.launch(realtime=True, browser="notebook")

SO101._T = np.array(
    [
        [1, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ]
)
puma._T = np.array(
    [
        [1, 0, 0, 0.5],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ]
)
panda._T = np.array(
    [
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ]
)
G4._T = np.array(
    [
        [1, 0, 0, 1.5],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ]
)
env.add(G4)
env.add(panda)
env.add(SO101)
env.add(puma)
# env.hold()
env.set_camera_pose([-0.5, -2, 0.5], [1, 0.0, 0.5])

In [11]:
import robosandbox.performance.workspace.WorkSpace as ws

workspace = ws(SO101)
global_mu = workspace.global_indice(method="yoshikawa", initial_samples=10000)
print("Global Indices (yoshikawa):", global_mu)
fig = workspace.plot(
    color="yoshikawa",
)


Global Indices (yoshikawa): 0.001059830007034206


In [3]:
# get the current working directory
pwd = os.getcwd()
tld = os.path.join(pwd, "data")

mockrobot = rsb.models.URDF.Model(
    name="mockrobot",
    file_path="GenericDH_6dof_11.urdf",
    tld=tld,
)
mockrobot.q = np.zeros(6)
# print(SO101)

env = swift.Swift()
env.launch(realtime=True, browser="notebook")
env.add(mockrobot)


0

In [9]:
from robosandbox.models.URDF.Generic import GenericDH
import numpy as np
import swift

robot = GenericDH(
    dofs=6,
    a=[0, -0.42500, -0.39225, 0, 0, 0],
    d=[0.089459, 0, 0, 0.10915, 0.09465, 0.0823],
    alpha=[np.pi / 2, 0, 0, np.pi / 2, -np.pi / 2, 0],
    offset=[0, 0, 0, 0, np.pi / 3, 0],
    link_radius=[0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
    actuator_radius=[0.02, 0.02, 0.02, 0.02, 0.01, 0.01],
)
robot.q = np.array([0, 0, 0, 0, np.pi / 3, 0])
env = swift.Swift()
env.launch(realtime=True, browser="notebook")
env.add(robot)
print(robot)


/var/folders/x1/r8g1dmlj1dn7lz6z4bdx_mfr0000gn/T/ipykernel_55797/1843727298.py:5: UserWarning:

Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.



ERobot: GenericDH (by Generic), 6 joints (RRRRRR), geometry
┌──────┬──────┬───────┬────────┬─────────────────────────────────────────────────────────────┐
│ link │ link │ joint │ parent │                     ETS: parent to link                     │
├──────┼──────┼───────┼────────┼─────────────────────────────────────────────────────────────┤
│    0 │ a0   │       │ BASE   │ SE3()                                                       │
│    1 │ l0   │     0 │ a0     │ SE3() ⊕ Rz(q0)                                              │
│    2 │ a1   │       │ l0     │ SE3()                                                       │
│    3 │ l1   │     1 │ a1     │ SE3(0, 0, 0.08946) ⊕ Ry(-q1)                                │
│    4 │ a2   │       │ l1     │ SE3()                                                       │
│    5 │ l2   │     2 │ a2     │ SE3(-0.425, 0, 0) ⊕ Ry(-q2)                                 │
│    6 │ a3   │       │ l2     │ SE3()                                               